### LSTM

In [30]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-11-30,96449.054688,86570.707812,91648.316009,82261.362670,68184.502386,65.500818,5191.352498,5794.364534,97279.789062,1
2024-12-01,97279.789062,87497.285156,92184.646776,82850.320568,68474.007726,66.656792,5022.591458,5640.009918,95865.304688,0
2024-12-02,95865.304688,88383.152865,92535.185625,83360.712102,68746.557944,62.798383,4720.297400,5456.067415,96002.164062,1


In [ ]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()
y.shape, y

((3533, 1),
 array([[1],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64))

In [90]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Ya lo convierte en un array de numpy 
X_scaled.shape, X_scaled

((3533, 5),
 array([[-0.94080889, -0.93947655, -0.45317299, -0.153607  , -0.1574351 ],
        [-0.94082848, -0.93945397, -0.10429159, -0.15279471, -0.15731524],
        [-0.94086202, -0.9394569 , -0.362657  , -0.15247791, -0.15715212],
        ...,
        [ 3.32887102,  3.49210365,  0.6301845 ,  3.92460032,  4.84291506],
        [ 3.37290181,  3.50796043,  0.64598744,  3.68314866,  4.65691486],
        [ 3.42290216,  3.53495947,  0.948451  ,  3.64078593,  4.49912575]]))

In [91]:
sequence_length = 30 # Ventana temporal de 30 dias 
X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X_scaled)): # X_scaled tiene dimension de 1000, X_sequences tendra 1000 - 30(sequence_leght)
    X_sequences.append(X_scaled[i - sequence_length: i])
    y_sequences.append(y[i])
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)
X_sequences.shape, y_sequences.shape


((3503, 30, 5), (3503, 1))

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.01, shuffle=False)

In [93]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore

model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=False), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# model.add(Bidirectional(LSTM(units=100, return_sequences=False)))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

model.add(Dense(units=15, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\ivoto\miniconda3\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [94]:
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5103 - loss: 0.7787 - val_accuracy: 0.4444 - val_loss: 0.7009
Epoch 2/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5411 - loss: 0.7116 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 3/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5026 - loss: 0.7093 - val_accuracy: 0.5833 - val_loss: 0.6791
Epoch 4/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5294 - loss: 0.7011 - val_accuracy: 0.5833 - val_loss: 0.6894
Epoch 5/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5143 - loss: 0.6950 - val_accuracy: 0.5833 - val_loss: 0.6813
Epoch 6/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5269 - loss: 0.6910 - val_accuracy: 0.5556 - val_loss: 0.6917
Epoch 7/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5284 - loss: 0.6954 - val_accuracy: 0.4722 - val_loss: 0.7169
Epoch 8/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5348 - loss: 0.6894 - val_accu

In [95]:
# 9. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo LSTM: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4317 - loss: 0.7003
Precisión del modelo LSTM: 0.44


In [25]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
              precision    recall  f1-score   support

           0       0.52      0.82      0.64        17
           1       0.67      0.32      0.43        19

    accuracy                           0.56        36
   macro avg       0.59      0.57      0.53        36
weighted avg       0.60      0.56      0.53        36

[[14  3]
 [13  6]]
